In [1]:
import graphlab
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1472050763.log


This non-commercial license of GraphLab Create for academic use is assigned to nirav_605@hotmail.com and will expire on August 19, 2017.


In [2]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])
# ignore all 3 star reviews
products = products[products['rating'] != 3]
products['sentiment'] = products['rating'] >= 4 ## 0 or 1
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


In [3]:
testdic = {'and':1, 'test':5, 'great':2, 'awesome':10}
testdic2 = {'and':1, 'test':5, 'great':2}

In [4]:
def awesome_count(dictionary):
    if 'awesome' in dictionary:
        return dictionary['awesome']
    else:
        return 0;

In [5]:
print awesome_count(testdic), awesome_count(testdic2)

10 0


In [6]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])  ## Creating word count

In [7]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [8]:
### add columns for each word and its count
for words in selected_words:
    def str_count(dictionary):
        if words in dictionary:
            return dictionary[words]
        else:
            return 0;
    products[words] = products['word_count'].apply(str_count)

In [9]:
## Dictionary of number of times "selected_word" was used.
Sum_SelWords = {word:0 for word in selected_words}
for word in selected_words:
#   print products[word].sum()
    Sum_SelWords[word] = products[word].sum()

In [10]:
print Sum_SelWords

{'fantastic': 873, 'love': 40277, 'bad': 3197, 'awesome': 2002, 'great': 42420, 'terrible': 673, 'amazing': 1305, 'horrible': 659, 'awful': 345, 'hate': 1057, 'wow': 131}


In [11]:
train_data, test_data = products.random_split(0.8,seed=0)

In [12]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.348442     | 0.844269          | 0.843142            |

| 5         | 6        | 1.427616     | 0.844269          | 0.843142            |

| 6         | 7        | 1.550842     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [13]:
#sf.sort('a')
coeff = selected_words_model['coefficients'];
coeff = coeff.sort('value', ascending = False)
coeff.print_rows(num_rows=12, num_columns=5)

+-------------+-------+-------+------------------+------------------+
|     name    | index | class |      value       |      stderr      |
+-------------+-------+-------+------------------+------------------+
|     love    |  None |   1   |  1.39989834302   | 0.0287147460124  |
| (intercept) |  None |   1   |  1.36728315229   | 0.00861805467824 |
|   awesome   |  None |   1   |  1.05800888878   |  0.110865296265  |
|   amazing   |  None |   1   |  0.892802422508  |  0.127989503231  |
|  fantastic  |  None |   1   |  0.891303090304  |  0.154532343591  |
|    great    |  None |   1   |  0.883937894898  | 0.0217379527921  |
|     wow     |  None |   1   | -0.0541450123333 |  0.275616449416  |
|     bad     |  None |   1   | -0.985827369929  | 0.0433603009142  |
|     hate    |  None |   1   |  -1.40916406276  | 0.0771983993506  |
|    awful    |  None |   1   |  -1.76469955631  |  0.134679803365  |
|   horrible  |  None |   1   |  -1.99651800559  | 0.0973584169028  |
|   terrible  |  Non

In [14]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376,
 'auc': 0.6648096413721418,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 |      1       |        0        |  130  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.914242563530107,
 'log_loss': 0.4054747110366022,
 'precision': 0.8453551912568306,
 'recall': 0.9953531598513011,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 

# 4. Interpreting the difference in performance between the models

In [15]:
#products.remove_columns(selected_words)
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']
#diaper_champ_reviews.head()

In [16]:
## Build the sentiment_model
train_data, test_data = products.random_split(0.8,seed=0)
sentiment_model = graphlab.logistic_classifier.create(train_data, 
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 4.631390     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 6.855198     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [17]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [18]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [19]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1, 'over': 1,'rweek': 1, 'sooo': 1, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2, 'just': 4,""don't"": 2, 'one,': 1, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1, 'able': 2,'over': 1, 'soon': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'son': 2, 'all': 1,'bags.': 1, 'son,': 1, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4, 'this': 3,'stink': 1, 'garbage' ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1, '-': 3,'both': 1, 'results': 1, ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'all': 1, 'humid': 1,'just': 1, 'less': 1, ...",1,0


In [20]:
## Predict sentiment of most positive review using only the selected words!
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290673]

In [21]:
diaper_champ_reviews[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'